In [1]:
import os
import cv2
import numpy as np
import joblib
from datetime import datetime
from mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder, Normalizer, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from ultralytics import YOLO
from collections import deque
import tensorflow as tf

In [2]:
DATASET_DIR = r"C:\Users\hp\project\cropped_dataset"
IMG_SIZE = (160, 160)
MARGIN = 20
AUG_PER_IMAGE = 3          
UNKNOWN_THRESHOLD = 0.60   

In [3]:
SVM_PATH = r"C:\Users\hp\project\svm_model.pkl"
ENCODER_PATH = r"C:\Users\hp\project\encoder.pkl"
SCALER_PATH = r"C:\Users\hp\project\scaler.pkl"
RF_PATH = r"C:\Users\hp\project\rf_model.pkl"
L2_FLAG_PATH = r"C:\Users\hp\project\l2_normalizer.note"
FACENET_NAME = r"C:\Users\hp\project\facenet_embedder.note"
YOLO_MODEL_PATH = r"C:\Users\hp\project\yolov8n.pt"

In [4]:
detector = MTCNN()
embedder = FaceNet()
yolo_model = YOLO("yolov8n.pt") 

Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.


In [11]:
def square_crop_with_margin(box, img_w, img_h, margin=MARGIN):
    x, y, w, h = box
    x, y = abs(x), abs(y)
    x1, y1 = max(0, x - margin), max(0, y - margin)
    x2, y2 = min(img_w, x + w + margin), min(img_h, y + h + margin)
    bw, bh = x2 - x1, y2 - y1
    side = max(bw, bh)
    cx = x1 + bw // 2
    cy = y1 + bh // 2
    x1 = max(0, cx - side // 2)
    y1 = max(0, cy - side // 2)
    x2 = min(img_w, x1 + side)
    y2 = min(img_h, y1 + side)
    return x1, y1, x2, y2

In [12]:
def align_face(image, box, landmarks=None):
    if landmarks is not None:
        left_eye = np.array(landmarks[0][0])  
        right_eye = np.array(landmarks[1][0])
        angle = np.arctan2(right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]) * 180 / np.pi
        M = cv2.getRotationMatrix2D((image.shape[1]//2, image.shape[0]//2), angle, 1.0)
        aligned = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
        x1, y1, x2, y2 = square_crop_with_margin(box, image.shape[1], image.shape[0])
        return aligned[y1:y2, x1:x2]
    else:
        x1, y1, x2, y2 = square_crop_with_margin(box, image.shape[1], image.shape[0])
        return image[y1:y2, x1:x2]

In [13]:
def extract_face(path, required_size=IMG_SIZE):
    img = cv2.imread(path)
    if img is None:
        print(f" Could not read image: {path}")
        return None
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(rgb)
    if not results:
        results_yolo = yolo_model(rgb)
        boxes = results_yolo[0].boxes
        if len(boxes) > 0:
            for box in boxes:
                if int(box.cls) == 0:  # Class 0: 'person'
                    box_coords = box.xyxy[0].cpu().numpy()
                    x1, y1, x2, y2 = map(int, box_coords)
                    face = rgb[y1:y2, x1:x2]
                    h, w = face.shape[:2]
                    center_x, center_y = w // 2, h // 2
                    angle = 0  
                    M = cv2.getRotationMatrix2D((center_x, center_y), angle, 1.0)
                    face = cv2.warpAffine(face, M, (w, h))
                    break
            else:
                print(f" No 'person' detected in fallback: {path}")
                return None
        else:
            print(f" No detection in fallback: {path}")
            return None
    else:
        box = results[0]['box']
        landmarks = [[results[0]['keypoints']['left_eye']], [results[0]['keypoints']['right_eye']]]
        face = align_face(rgb, box, landmarks)
    if face.size == 0:
        return None
    face = cv2.resize(face, required_size)
    face = face.astype("float32") / 255.0
    return face

In [14]:
def augment_face(face_rgb_01):
    aug_samples = []

    flipped = cv2.flip(face_rgb_01, 1)
    aug_samples.append(flipped)
    
    beta = np.random.uniform(-30, 30)
    bgr = cv2.cvtColor((face_rgb_01*255).astype(np.uint8), cv2.COLOR_RGB2BGR)
    bright = cv2.convertScaleAbs(bgr, alpha=1.0, beta=beta)
    bright = cv2.cvtColor(bright, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0
    aug_samples.append(bright)

    angle = np.random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D((IMG_SIZE[0]//2, IMG_SIZE[1]//2), angle, 1.0)
    rot = cv2.warpAffine((face_rgb_01*255).astype(np.uint8), M, IMG_SIZE)
    rot = rot.astype(np.float32)/255.0
    aug_samples.append(rot)

    blur = cv2.GaussianBlur((face_rgb_01*255).astype(np.uint8), (3,3), 0)
    blur = blur.astype(np.float32)/255.0
    aug_samples.append(blur)

    shear = np.random.uniform(-0.1, 0.1)
    M = np.array([[1, shear, 0], [0, 1, 0]], dtype=np.float32)
    shear_img = cv2.warpAffine((face_rgb_01*255).astype(np.uint8), M, IMG_SIZE)
    shear_img = shear_img.astype(np.float32)/255.0
    aug_samples.append(shear_img)
    np.random.shuffle(aug_samples)
    return aug_samples[:AUG_PER_IMAGE]

In [15]:
def face_to_embedding(face_rgb_01):
    face_255 = (face_rgb_01*255).astype(np.uint8)
    face_255 = np.expand_dims(face_255, 0)
    embs = embedder.embeddings(face_255)
    return embs[0]

In [16]:
def load_models(svm_path=SVM_PATH, rf_path=RF_PATH, encoder_path=ENCODER_PATH, scaler_path=SCALER_PATH):
    if not (os.path.exists(svm_path) and os.path.exists(rf_path) and os.path.exists(encoder_path) and os.path.exists(scaler_path)):
        print(f" Model files missing: {svm_path}, {rf_path}, {encoder_path}, {scaler_path}")
        return None, None, None, None, None
    try:
        svm = joblib.load(svm_path)
        rf = joblib.load(rf_path)
        encoder = joblib.load(encoder_path)
        scaler = joblib.load(scaler_path)
        l2 = Normalizer(norm="l2")
        print(f"Loaded SVM, RF, encoder, scaler, and L2 normalizer")
        return svm, rf, encoder, scaler, l2
    except Exception as e:
        print(f"Error loading models: {str(e)}")
        return None, None, None, None, None

In [17]:
def realtime_recognition():
    svm, rf, encoder, scaler, l2 = load_models()
    if svm is None:
        return
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return
    win_name = "Real-Time Face Recognition (Press 'q' to quit)"
    pred_queues = {}  
    MIN_QUEUE_SIZE = 2  

    def match_box(new_box, old_boxes):
        if not old_boxes:
            return None
        nx1, ny1, nw, nh = new_box
        new_center = (nx1 + nw / 2, ny1 + nh / 2)
        min_dist = float('inf')
        matched_key = None
        for old_box in old_boxes:
            ox1, oy1, ow, oh = old_box
            old_center = (ox1 + ow / 2, oy1 + oh / 2)
            dist = ((new_center[0] - old_center[0])**2 + (new_center[1] - old_center[1])**2)**0.5
            if dist < min_dist and dist < 100:  
                min_dist = dist
                matched_key = tuple(old_box)
        return matched_key

    last_boxes = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print(" Failed to grab frame")
                break
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces = []
            boxes_to_display = []
            results = detector.detect_faces(rgb)
            if results:
                for res in results:
                    box = res['box']
                    landmarks = [[res['keypoints']['left_eye']], [res['keypoints']['right_eye']]]
                    face = align_face(rgb, box, landmarks)
                    if face.size == 0:
                        continue
                    faces.append((face, box))
                    boxes_to_display.append(box)
            else:
                results_yolo = yolo_model(rgb)
                boxes = results_yolo[0].boxes
                for box in boxes:
                    if int(box.cls) != 0:  
                        continue
                    box_coords = box.xyxy[0].cpu().numpy()
                    x1, y1, x2, y2 = map(int, box_coords)
                    face = align_face(rgb, [x1, y1, x2-x1, y2-y1])
                    if face.size == 0:
                        continue
                    faces.append((face, [x1, y1, x2-x1, y2-y1]))
                    boxes_to_display.append([x1, y1, x2-x1, y2-y1])
            last_boxes = boxes_to_display
            if not faces:
                cv2.imshow(win_name, frame)
                continue
   
            for face, box in faces:
                face = cv2.resize(face, IMG_SIZE).astype(np.float32) / 255.0
                emb = face_to_embedding(face).reshape(1, -1)
                emb = l2.transform(emb)
                emb = scaler.transform(emb)
                probs_svm = svm.predict_proba(emb)[0]
                probs_rf = rf.predict_proba(emb)[0]
                probs = (probs_svm + probs_rf) / 2
                pred_idx = np.argmax(probs)
                pred_conf = probs[pred_idx]
                box_key = match_box(box, last_boxes)
                
                if box_key is None:
                    box_key = tuple(box)
                if box_key not in pred_queues:
                    pred_queues[box_key] = deque(maxlen=5)
                pred_queues[box_key].append((pred_idx, pred_conf))
                if len(pred_queues[box_key]) >= MIN_QUEUE_SIZE:
                    avg_conf = np.mean([conf for _, conf in pred_queues[box_key]])
                    mode_idx = max(set([idx for idx, _ in pred_queues[box_key]]), key=[idx for idx, _ in pred_queues[box_key]].count)
                    pred_name = encoder.inverse_transform([mode_idx])[0]
                    display = f"{pred_name} ({avg_conf*100:.1f}%)" if avg_conf >= UNKNOWN_THRESHOLD else f"Unknown ({avg_conf*100:.1f}%)"
                    color = (0, 255, 0) if avg_conf >= UNKNOWN_THRESHOLD else (0, 0, 255)
                else:
                    pred_name = encoder.inverse_transform([pred_idx])[0]
                    display = f"{pred_name} ({pred_conf*100:.1f}%)" if pred_conf >= UNKNOWN_THRESHOLD else f"Unknown ({pred_conf*100:.1f}%)"
                    color = (0, 255, 0) if pred_conf >= UNKNOWN_THRESHOLD else (0, 0, 255)
                x1, y1, x2, y2 = square_crop_with_margin(box, rgb.shape[1], rgb.shape[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, display, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                
            active_keys = [tuple(box) for box in boxes_to_display]
            pred_queues = {k: q for k, q in pred_queues.items() if k in active_keys}
            cv2.imshow(win_name, frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("👋 Closing camera...")
                break
            if cv2.getWindowProperty(win_name, cv2.WND_PROP_VISIBLE) < 1:
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)

In [19]:
svm, rf, encoder, scaler, l2 = load_models()
realtime_recognition()

Loaded SVM, RF, encoder, scaler, and L2 normalizer
Loaded SVM, RF, encoder, scaler, and L2 normalizer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━